# Project Proposal

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Possible questions for Starbucks data:
* Classification: Can we use selected nutritional values of a Starbucks drink to predict its beverage category or beverage type?
* Regression: Can we use the total fat in a Starbucks drink to predict its calories?
* Regression: Can we use the total carbs in a Starbucks food item to predict its calories?

Possible questions for Salaries data:
* Classification: Can we use starting median salary and mid-career median salary to predict the region of education?
* Regression: Can we use starting median salary to predict mid-career median salary?

In [5]:
drink_data <- read_csv("data/starbucks/drinkMenu.csv")
food_data <- read_csv("data/starbucks/foodMenu.csv")
food_data

Parsed with column specification:
cols(
  Beverage_category = col_character(),
  Beverage = col_character(),
  Beverage_prep = col_character(),
  Calories = col_double(),
  `Total Fat (g)` = col_character(),
  `Trans Fat (g)` = col_double(),
  `Saturated Fat (g)` = col_double(),
  `Sodium (mg)` = col_double(),
  `Total Carbohydrates (g)` = col_double(),
  `Cholesterol (mg)` = col_double(),
  `Dietary Fibre (g)` = col_double(),
  `Sugars (g)` = col_double(),
  `Protein (g)` = col_double(),
  `Vitamin A (% DV)` = col_character(),
  `Vitamin C (% DV)` = col_character(),
  `Calcium (% DV)` = col_character(),
  `Iron (% DV)` = col_character(),
  `Caffeine (mg)` = col_character()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  Calories = col_double(),
  `Fat (g)` = col_double(),
  `Carb. (g)` = col_double(),
  `Fiber (g)` = col_double(),
  `Protein (g)` = col_double()
)



X1,Calories,Fat (g),Carb. (g),Fiber (g),Protein (g)
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Chonga Bagel,300,5.0,50,3,12
8Grain Roll,380,6.0,70,7,10
Almond Croissant,410,22.0,45,3,10
Apple Fritter,460,23.0,56,2,7
Banana Nut Bread,420,22.0,52,2,6
Blueberry Muffin with Yogurt and Honey,380,16.0,53,1,6
Blueberry Scone,420,17.0,61,2,5
Butter Croissant,240,12.0,28,1,5
Butterfly Cookie,350,22.0,38,0,2


In [9]:
byRegion <- read_csv("data/salaries/byRegion.csv")
byCollege <- read_csv("data/salaries/byCollege.csv")
byDegree <- read_csv("data/salaries/byDegree.csv")
byCollege

Parsed with column specification:
cols(
  `School Name` = col_character(),
  Region = col_character(),
  `Starting Median Salary` = col_character(),
  `Mid-Career Median Salary` = col_character(),
  `Mid-Career 10th Percentile Salary` = col_character(),
  `Mid-Career 25th Percentile Salary` = col_character(),
  `Mid-Career 75th Percentile Salary` = col_character(),
  `Mid-Career 90th Percentile Salary` = col_character()
)

Parsed with column specification:
cols(
  `School Name` = col_character(),
  `School Type` = col_character(),
  `Starting Median Salary` = col_character(),
  `Mid-Career Median Salary` = col_character(),
  `Mid-Career 10th Percentile Salary` = col_character(),
  `Mid-Career 25th Percentile Salary` = col_character(),
  `Mid-Career 75th Percentile Salary` = col_character(),
  `Mid-Career 90th Percentile Salary` = col_character()
)

Parsed with column specification:
cols(
  `Undergraduate Major` = col_character(),
  `Starting Median Salary` = col_character(),
  `Mid-Car

School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Massachusetts Institute of Technology (MIT),Engineering,"$72,200.00","$126,000.00","$76,800.00","$99,200.00","$168,000.00","$220,000.00"
California Institute of Technology (CIT),Engineering,"$75,500.00","$123,000.00",N/A,"$104,000.00","$161,000.00",N/A
Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",N/A,"$96,000.00","$180,000.00",N/A
"Polytechnic University of New York, Brooklyn",Engineering,"$62,400.00","$114,000.00","$66,800.00","$94,300.00","$143,000.00","$190,000.00"
Cooper Union,Engineering,"$62,200.00","$114,000.00",N/A,"$80,200.00","$142,000.00",N/A
Worcester Polytechnic Institute (WPI),Engineering,"$61,000.00","$114,000.00","$80,000.00","$91,200.00","$137,000.00","$180,000.00"
Carnegie Mellon University (CMU),Engineering,"$61,800.00","$111,000.00","$63,300.00","$80,100.00","$150,000.00","$209,000.00"
Rensselaer Polytechnic Institute (RPI),Engineering,"$61,100.00","$110,000.00","$71,600.00","$85,500.00","$140,000.00","$182,000.00"
Georgia Institute of Technology,Engineering,"$58,300.00","$106,000.00","$67,200.00","$85,200.00","$137,000.00","$183,000.00"
